In [66]:
from boxoffice.db.db import (
    Distributor,
    Franchise,
    MovieDistributor,
    MovieFranchise,
    sqlite_db_connect,
    Movie,
    BoxOfficeDay,
)
from pandera.typing import DataFrame
import datetime
from peewee import fn, JOIN
from boxoffice.db.frames import MovieCompleteSchema, get_box_office_day_frame, JoinedMovieSchema

In [67]:

# use a subquery to get release dates for non_preview days
subquery = (
    BoxOfficeDay.select(
        BoxOfficeDay.movie_id,
        fn.MIN(BoxOfficeDay.date).alias("release_day_non_preview"),
        fn.MAX(BoxOfficeDay.theaters).alias("largest_theater_count"),
    )
    .where(BoxOfficeDay.is_preview == False)
    .group_by(BoxOfficeDay.movie)
)

days_over_1000_theaters_query = (
    BoxOfficeDay.select(
        BoxOfficeDay.movie_id,
        fn.COUNT(BoxOfficeDay.theaters).alias("days_over_1000_theaters"),
    )
    .where(BoxOfficeDay.theaters >= 1000)
    .group_by(BoxOfficeDay.movie)
)

days_over_1000000_revenue_query = (
    BoxOfficeDay.select(
        BoxOfficeDay.movie_id,
        fn.COUNT(BoxOfficeDay.revenue).alias("days_over_1000000_revenue"),
    )
    .where(BoxOfficeDay.revenue >= 1000000)
    .group_by(BoxOfficeDay.movie)
)

days_over_100000_revenue_query = (
    BoxOfficeDay.select(
        BoxOfficeDay.movie_id,
        fn.COUNT(BoxOfficeDay.revenue).alias("days_over_100000_revenue"),
    )
    .where(BoxOfficeDay.revenue >= 100000)
    .group_by(BoxOfficeDay.movie)
)

preview_sum_query = (
    BoxOfficeDay.select(
        BoxOfficeDay.movie_id,
        fn.SUM(BoxOfficeDay.revenue).alias("preview_sum"),
    )
    .where(BoxOfficeDay.is_preview == True)
    .group_by(BoxOfficeDay.movie)
)

movies = (
    Movie.select(
        Movie.id,
        Movie.truncated_title,
        Movie.slug,
        Movie.title,
        Movie.release_year,
        Movie.mpaa_rating,
        Movie.running_time,
        Movie.synopsis,
        Movie.mpaa_rating_reason,
        Movie.budget,
        Movie.creative_type,
        Movie.genre,
        Movie.production_method,
        Movie.source,
        fn.SUM(BoxOfficeDay.revenue).alias("total_box_office"),
        fn.MIN(BoxOfficeDay.date).alias("release_day"),
        MovieDistributor.distributor.alias("distributor_id"),
        MovieFranchise.franchise.alias("franchise_id"),
        Distributor.name.alias("distributor_name"),
        Distributor.slug.alias("distributor_slug"),
        Franchise.name.alias("franchise_name"),
        Franchise.slug.alias("franchise_slug"),
        subquery.c.release_day_non_preview,
        subquery.c.largest_theater_count,
        days_over_1000_theaters_query.c.days_over_1000_theaters,
        days_over_1000000_revenue_query.c.days_over_1000000_revenue,
        days_over_100000_revenue_query.c.days_over_100000_revenue,
        preview_sum_query.c.preview_sum,
    )
    .join(BoxOfficeDay, on=(Movie.id == BoxOfficeDay.movie))
    .group_by(Movie.id)
    .join_from(Movie, MovieDistributor)
    .join_from(Movie, MovieFranchise, JOIN.LEFT_OUTER)
    .join_from(MovieDistributor, Distributor)
    .join_from(MovieFranchise, Franchise, JOIN.LEFT_OUTER)
    .join(subquery, on=(Movie.id == subquery.c.movie_id))
    .join(
        days_over_1000_theaters_query,
        on=(Movie.id == days_over_1000_theaters_query.c.movie_id),
    )
    .join(
        days_over_1000000_revenue_query,
        on=(Movie.id == days_over_1000000_revenue_query.c.movie_id),
    )
    .join(
        days_over_100000_revenue_query,
        on=(Movie.id == days_over_100000_revenue_query.c.movie_id),
    )
    .join(
        preview_sum_query,
        JOIN.LEFT_OUTER,
        on=(Movie.id == preview_sum_query.c.movie_id),
    )
)

dicts = movies.dicts()

# print the beginning of dicts
print(dicts)

# make sure the release_day_non_preview is a date
for entry in dicts:
    entry["release_day_non_preview"] = datetime.datetime.strptime(
        entry["release_day_non_preview"], "%Y-%m-%d"
    ).date()

movies_df = DataFrame[JoinedMovieSchema](dicts)

# within synopsis, replace commas and newlines
movies_df["synopsis"] = movies_df["synopsis"].str.replace(",", "%2C")
movies_df["synopsis"] = movies_df["synopsis"].str.replace("\n", "%0A")

kept_ids = movies_df["id"]

print(
    f"There are {len(kept_ids)} movies in the movie dataframe, now filtering out the box office days"
)

# need to calculate the release day of the week
bodf = get_box_office_day_frame()

if bodf is None:
    print("No box office days found")
    exit()

# now filter out the box office days for movies that are not in the movie dataframe
prior_len = len(bodf)
bodf = bodf[bodf["movie"].isin(kept_ids)]

# bodf = bodf.sort_values("date").sort_values("movie")
bodf.reset_index(drop=True, inplace=True)
# bodf = bodf.sort_values("date").sort_values("movie")

print(
    f"Filtered out {prior_len - len(bodf)} box office days, there are now {len(bodf)} box office days"
)

# add the columns to the df
# copy the movies_df but without any typing
df = movies_df.copy()

# df.reset_index(drop=True, inplace=True)

# print the total amount of dates
print(len(bodf["date"]))

dates = bodf.groupby("movie")["date"]

# print the na count of the dates
print(len(dates))

df["release_day_of_week"] = df["release_day"].apply(lambda x: x.weekday())

df["release_day_of_week_non_preview"] = df["release_day_non_preview"].apply(
    lambda x: x.weekday()
)

df["release_month"] = df["release_day_non_preview"].apply(lambda x: x.month)

df["release_day_of_month"] = df["release_day_non_preview"].apply(lambda x: x.day)

# get the opening weekend revenue for each movie. This is the sum of the first Friday, Saturday, and Sunday plus Thursday if there was a preview. Movies may not open on a Friday, so need to specifically get the first of each of these
# first_five_of_each_day = []
# first_each_day = []
# total_each_day = []
# is_preview = []
# for weekday in range(7):
#     days = (
#         bodf.where(bodf["day_of_week"] == weekday)
#         .sort_values("date")
#         .groupby("movie")
#     )

#     first_five_of_each_day.append(days["revenue"].head(5).sum())
#     first_each_day.append(days["revenue"].head(1).sum())
#     total_each_day.append(days["revenue"].sum())
#     is_preview.append(days["is_preview"].head(1).sum())

SELECT "t1"."id", "t1"."truncated_title", "t1"."slug", "t1"."title", "t1"."release_year", "t1"."mpaa_rating", "t1"."running_time", "t1"."synopsis", "t1"."mpaa_rating_reason", "t1"."budget", "t1"."creative_type", "t1"."genre", "t1"."production_method", "t1"."source", SUM("t2"."revenue") AS "total_box_office", MIN("t2"."date") AS "release_day", "t3"."distributor_id" AS "distributor_id", "t4"."franchise_id" AS "franchise_id", "t5"."name" AS "distributor_name", "t5"."slug" AS "distributor_slug", "t6"."name" AS "franchise_name", "t6"."slug" AS "franchise_slug", "t7"."release_day_non_preview", "t7"."largest_theater_count", "t8"."days_over_1000_theaters", "t9"."days_over_1000000_revenue", "t10"."days_over_100000_revenue", "t11"."preview_sum" FROM "movie" AS "t1" INNER JOIN "boxofficeday" AS "t2" ON ("t1"."id" = "t2"."movie_id") INNER JOIN "moviedistributor" AS "t3" ON ("t3"."movie_id" = "t1"."id") LEFT OUTER JOIN "moviefranchise" AS "t4" ON ("t4"."movie_id" = "t1"."id") INNER JOIN "distributo

In [68]:
if bodf is None:
    print("No box office days found")
    exit()

first_five_by_weekday = []
first_by_weekday = []
total_by_weekday = []
is_preview_by_weekday = []

for weekday in range(7):
    days = bodf[bodf["day_of_week"] == weekday].sort_values("date").reset_index(drop=True).groupby("movie")

    first_five_by_weekday.append(days.apply(lambda x: x.head(5)["revenue"].sum(),include_groups=False))
    first_by_weekday.append(days.apply(lambda x: x.head(1)["revenue"].sum(),include_groups=False))
    total_by_weekday.append(days.apply(lambda x: x["revenue"].sum(),include_groups=False))
    is_preview_by_weekday.append(days.apply(lambda x: x.head(1)["is_preview"].sum(),include_groups=False))

# bodf_sorted = bodf.sort_values(["movie", "date"], ascending=True).groupby(["movie", "day_of_week"])

# first_five_of_each_day = (
#     bodf_sorted
#     .apply(lambda x: x.head(5)["revenue"].sum(),include_groups=False)
#     .groupby("movie")
# )

# print("Printing first five of each day")
# print(first_five_of_each_day)

# first_each_day = (
#     bodf_sorted
#     .apply(lambda x: x.head(1)["revenue"].sum(),include_groups=False)
#     .groupby("movie")
# )
# total_each_day = (
#     bodf_sorted
#     .apply(lambda x: x["revenue"].sum(),include_groups=False)
#     .groupby("movie")
# )

# is_preview = (
#     bodf_sorted
#     .apply(lambda x: x.head(1)["is_preview"].sum(),include_groups=False)
#     .groupby("movie")
# )

In [69]:
# opening weekend revenue is the sum of the first Friday, Saturday, and Sunday plus Thursday if there was a preview
# opening_weekend_revenue = (
#     first_each_day.where(first_each_day["day_of_week"] == 4)
#     + first_each_day.where(first_each_day["day_of_week"] == 5)
#     + first_each_day.where(first_each_day["day_of_week"] == 6)
#     + first_each_day.where(first_each_day["day_of_week"] == 3).where(is_preview)
# )

opening_weekend_revenue = (
    first_by_weekday[4]
    + first_by_weekday[5]
    + first_by_weekday[6]
).reset_index(drop=True)

print("Printing opening weekend revenue")
print(opening_weekend_revenue)

# print the amount of NaNs in the opening weekend revenue
print(opening_weekend_revenue.isna().sum()) 

# replace NaNs with 0
opening_weekend_revenue = opening_weekend_revenue.fillna(0)

Printing opening weekend revenue
0       211435291.0
1        81251415.0
2        75009210.0
3        15454146.0
4       154201673.0
           ...     
2570      4544063.0
2571      2697850.0
2572            NaN
2573            NaN
2574            NaN
Length: 2575, dtype: float64
20


In [80]:
if bodf is None:
    print("No box office days found")
    exit()
# get the first five days of revenue for each movie

df = df.reset_index(drop=True).sort_index()


first_five_days = (
    bodf.where(bodf["is_preview"] == False)
    .sort_values("date")
    .groupby("movie")
    .apply(lambda x: x.head(5)["revenue"].sum(), include_groups=False)
    .reset_index(drop=True)
)
# get the first seven days of revenue for each movie
first_seven_days = (
    bodf.where(bodf["is_preview"] == False)
    .sort_values("date")
    .groupby("movie")
    .apply(lambda x: x.head(7)["revenue"].sum(), include_groups=False)
    .reset_index(drop=True)
)

print("Printing first five days of revenue")
print(first_five_days)

# get the total revenue within 365 days of release for each movie
total_revenue_within_365_days = (
    bodf.where(bodf["date"] <= bodf["date"] + datetime.timedelta(days=365))
    .groupby("movie")["revenue"]
    .sum()
    .reset_index(drop=True)
)

# get the length of total revenue within 365 days
print(len(total_revenue_within_365_days))

# get the opening weekend to total ratio for each movie
opening_weekend_to_total_ratio = opening_weekend_revenue / total_revenue_within_365_days
opening_weekend_to_total_ratio = opening_weekend_to_total_ratio.reset_index(drop=True)

# now get the ratios for each day of the week
first_five_ratios = []

for i in range(7):
    first_five_ratios.append(
        (first_five_by_weekday[i] / first_five_by_weekday[i - 1]).reset_index(drop=True)
    )

# now get the ratios for each day of the week
ratios = []

for i in range(7):
    ratios.append((total_by_weekday[i] / total_by_weekday[i - 1]).reset_index(drop=True))

# now we are done
df["opening_weekend_revenue"] = opening_weekend_revenue.fillna(0)
df["first_five_days_revenue"] = first_five_days
df["first_seven_days_revenue"] = first_seven_days
df["total_revenue_within_365_days"] = total_revenue_within_365_days
df["opening_weekend_to_total_ratio"] = opening_weekend_to_total_ratio
df["preview_to_weekend_ratio"] = df["preview_sum"] / opening_weekend_revenue
df["fri_sat_ratio_first_five"] = first_five_ratios[5]
df["sat_sun_ratio_first_five"] = first_five_ratios[6]
df["sun_mon_ratio_first_five"] = first_five_ratios[0]
df["mon_tue_ratio_first_five"] = first_five_ratios[1]
df["tue_wed_ratio_first_five"] = first_five_ratios[2]
df["wed_thu_ratio_first_five"] = first_five_ratios[3]
df["thu_fri_ratio_first_five"] = first_five_ratios[4]
df["fri_sat_ratio"] = ratios[5]
df["sat_sun_ratio"] = ratios[6]
df["sun_mon_ratio"] = ratios[0]
df["mon_tue_ratio"] = ratios[1]
df["tue_wed_ratio"] = ratios[2]
df["wed_thu_ratio"] = ratios[3]
df["thu_fri_ratio"] = ratios[4]

df.to_csv("movies.csv")

typed = DataFrame[MovieCompleteSchema](df)

Printing first five days of revenue
0       261190114.0
1       103054855.0
2       122609535.0
3        19348423.0
4       205448790.0
           ...     
2575      5590906.0
2576      3391211.0
2577      9559726.0
2578      1331864.0
2579      1660092.0
Length: 2580, dtype: float64
2580


SchemaError: non-nullable series 'opening_weekend_revenue' contains null values:
2575   NaN
2576   NaN
2577   NaN
2578   NaN
2579   NaN
Name: opening_weekend_revenue, dtype: float64